<a href="https://colab.research.google.com/github/martinpius/Applied-Predictive-Modeling2/blob/master/BRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
from google.colab import drive
try:
  drive.mount("/content/drive", force_remount = True)
  COLAB = True
  import tensorflow
  print(f"You are using google colab with tensorflow version: {tensorflow.__version__}")
except:
  COLAB = False
  print('Not using Google Colab')
  

Mounted at /content/drive
You are using google colab with tensorflow version: 2.3.0


In [102]:
def time_setting(x):
  hours = int(x/(60 * 60))
  minutes = int(x % (60 * 60)/60)
  seconds = int(x % 60)
  return f"{hours}: {minutes:>02}: {seconds:>05.2f}"

In [134]:
#Loading the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn import metrics
from sklearn.model_selection import train_test_split
import tensorflow.keras
from tensorflow.keras.layers import Bidirectional, BatchNormalization, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Activation, Dropout



In [104]:
#Loading data 
if COLAB:
  path = "/content/drive/My Drive/fake-news"
else:
  pass
train_path = "/content/drive/My Drive/fake-news/train.csv"
test_path = "/content/drive/My Drive/fake-news/test.csv"
submit_path = "/content/drive/My Drive/fake-news/submit.csv"
train_data = pd.read_csv(train_path, na_values = ['NA', '?'])
test_data = pd.read_csv(test_path, na_values = ['NA','?'])
submit = pd.read_csv(submit_path, na_values = ['NA', '?'])

In [105]:
display(train_data.head(10))

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [106]:
display(submit.head(10))

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1
5,20805,1
6,20806,1
7,20807,1
8,20808,0
9,20809,1


In [107]:
display(test_data.head(10))

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
5,20805,Trump is USA's antique hero. Clinton will be n...,NaN,Trump is USA's antique hero. Clinton will be n...
6,20806,Pelosi Calls for FBI Investigation to Find Out...,Pam Key,"Sunday on NBC’s “Meet the Press,” House Minori..."
7,20807,Weekly Featured Profile – Randy Shannon,Trevor Loudon,You are here: Home / *Articles of the Bound* /...
8,20808,Urban Population Booms Will Make Climate Chang...,NaN,Urban Population Booms Will Make Climate Chang...
9,20809,NaN,cognitive dissident,don't we have the receipt?


In [108]:
train_data.isna().value_counts()

id     title  author  text   label
False  False  False   False  False    18285
              True    False  False     1918
       True   False   False  False      558
       False  True    True   False       39
dtype: int64

In [109]:
train_data.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [110]:
#Clear the missing values...Drop the rows (since we are dealing with the textual doc)
train_data.dropna(axis = 0, inplace = True)

In [111]:
train_data.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [112]:
train_data.shape

(18285, 5)

In [113]:
#Get the predictors and the labels and the pre-process the textual data
label = train_data['label']

In [114]:
x = train_data.drop(columns = ['label'], axis = 1)

In [115]:
x.columns

Index(['id', 'title', 'author', 'text'], dtype='object')

In [116]:
news = x.copy()

In [117]:
news.reset_index(inplace=True)

In [118]:
cov = news['title']

In [119]:
cov[0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [120]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 

In [121]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [122]:
pts = PorterStemmer()
bag = []
for i in range(0,len(news)):
  mesage = re.sub('[^a-zA-Z]', ' ', news['title'][i])#Get rid-off all characters apart from common alphabets
  mesage = mesage.lower()#Change the document into lower case
  mesage = mesage.split()#Get the list of words for each title.
  mesage = [pts.stem(word) for word in mesage if not word in stopwords.words('english')]#stem the document
  mesage = ' '.join(mesage)
  bag.append(mesage)

In [123]:
display(bag[0:10])

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [124]:
size_voc = 10000 #Assume a dictionary of 10000 words
sent_len = 20 #Assume no sentence will have more then 20 words
#get the one-hot representantion for each title:
indexes = [one_hot(words, size_voc) for words in bag]

In [125]:
indexes[0:50]

[[1051, 370, 8318, 464, 3922, 9433, 8228, 7304, 9074, 1169],
 [2530, 2879, 7849, 9171, 3092, 8282, 874],
 [8352, 3632, 4363, 5236],
 [6175, 3592, 4505, 3752, 1767, 812],
 [2497, 3092, 5935, 9435, 8501, 8681, 3092, 9380, 1685, 5880],
 [2682,
  6931,
  6558,
  1613,
  2381,
  8967,
  1988,
  195,
  5343,
  987,
  7649,
  7025,
  4546,
  7677,
  874],
 [7284, 74, 7202, 7502, 2255, 3904, 7663, 5999, 7313, 4044, 7639],
 [4748, 482, 6563, 7773, 8270, 6332, 8967, 1009, 7313, 4044, 7639],
 [1812, 6691, 1257, 2510, 9393, 1787, 6979, 2608, 8967, 1548],
 [7023, 8641, 4654, 4933, 1542, 5664, 9976, 9545],
 [5064, 7712, 771, 1888, 2689, 9558, 9120, 5577, 1418, 1084, 4588],
 [3752, 4338, 3922, 1787, 8967, 8270],
 [9893, 9427, 1767, 5075, 693, 4593, 2264, 6360, 7263],
 [8315, 2338, 6122, 599, 294, 3035, 5019, 7313, 4044, 7639],
 [6422, 3690, 7614, 7411, 6810, 7313, 4044, 7639],
 [2367, 3470, 3624, 8590, 1442, 9394, 7916, 4279, 950, 1654],
 [7901, 3380, 2879],
 [2388, 485, 8461, 1966, 8967, 4174, 9894,

In [126]:
#Padd the one_hot vectors to have common dimension
indexes_padded = pad_sequences(indexes, maxlen = sent_len, padding = 'pre')

In [127]:
indexes_padded[0:20]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1051,
         370, 8318,  464, 3922, 9433, 8228, 7304, 9074, 1169],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 2530, 2879, 7849, 9171, 3092, 8282,  874],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 8352, 3632, 4363, 5236],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 6175, 3592, 4505, 3752, 1767,  812],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2497,
        3092, 5935, 9435, 8501, 8681, 3092, 9380, 1685, 5880],
       [   0,    0,    0,    0,    0, 2682, 6931, 6558, 1613, 2381, 8967,
        1988,  195, 5343,  987, 7649, 7025, 4546, 7677,  874],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0, 7284,   74,
        7202, 7502, 2255, 3904, 7663, 5999, 7313, 4044, 7639],
       [   0,    0,    0,    0,    0,    

In [128]:
#Build RNN with words Embedding
features = 50
model1 = Sequential()
model1.add(Embedding(input_dim= size_voc, output_dim = features, input_length= sent_len))
model1.add(LSTM(units = 256, kernel_initializer = 'random_normal',dropout = 0.2, recurrent_dropout=0.2,
                return_sequences = True))
model1.add(BatchNormalization())
model1.add(Dropout(0.25))
model1.add(LSTM(128, kernel_initializer = 'random_normal', dropout = 0.1, recurrent_dropout=0.20,
                return_sequences = True))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation = 'sigmoid'))
model1.compile(loss = 'binary_crossentropy', metrics = ['accuracy'], optimizer = 'Adam')
model1.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 20, 50)            500000    
_________________________________________________________________
lstm_12 (LSTM)               (None, 20, 256)           314368    
_________________________________________________________________
batch_normalization_7 (Batch (None, 20, 256)           1024      
_________________________________________________________________
dropout_7 (Dropout)          (None, 20, 256)           0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 20, 128)           197120    
_________________________________________________________________
dropout_8 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 20, 1)            

In [129]:
#Building BRNN with word Embedding
model2 = Sequential()
model2.add(Embedding(input_dim = size_voc, output_dim = features, input_length=sent_len))
model2.add(Bidirectional(LSTM(units = 256, dropout= 0.2, recurrent_dropout=0.2,return_sequences=True)))
model2.add(BatchNormalization())
model2.add(Dropout(0.2))
model2.add(LSTM(128))
model2.add(Dense(1, activation= 'sigmoid'))
model2.compile(loss = 'binary_crossentropy', metrics = ['accuracy'], optimizer = 'Adam')
model2.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 20, 50)            500000    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 20, 512)           628736    
_________________________________________________________________
batch_normalization_8 (Batch (None, 20, 512)           2048      
_________________________________________________________________
dropout_9 (Dropout)          (None, 20, 512)           0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 128)               328192    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 1,459,105
Trainable params: 1,458,081
Non-trainable params: 1,024
________________________________________

In [130]:
x = np.array(indexes_padded)
y = np.array(label)

In [132]:
x_train, x_test,y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 123)

In [133]:
info = EarlyStopping(monitor = 'val_loss', patience = 20, restore_best_weights=True, verbose=1,min_delta=1e-3,
                     mode = 'auto')

In [135]:
model1.fit(x_train, y_train, validation_data=(x_test, y_test), 
           batch_size= 64,epochs = 1000, callbacks = [info], verbose= 2)
model2.fit(x_train, y_train, validation_data=(x_test, y_test), 
           verbose= 2, epochs = 1000, batch_size= 64, callbacks = [info])


Epoch 1/1000
229/229 - 40s - loss: 0.5876 - accuracy: 0.6545 - val_loss: 0.6593 - val_accuracy: 0.5766
Epoch 2/1000
229/229 - 40s - loss: 0.5302 - accuracy: 0.6926 - val_loss: 0.5915 - val_accuracy: 0.6567
Epoch 3/1000
229/229 - 42s - loss: 0.5032 - accuracy: 0.7075 - val_loss: 0.5939 - val_accuracy: 0.6647
Epoch 4/1000
229/229 - 40s - loss: 0.4848 - accuracy: 0.7153 - val_loss: 0.6183 - val_accuracy: 0.6284
Epoch 5/1000
229/229 - 39s - loss: 0.4730 - accuracy: 0.7212 - val_loss: 0.6214 - val_accuracy: 0.6717
Epoch 6/1000
229/229 - 39s - loss: 0.4624 - accuracy: 0.7263 - val_loss: 0.6457 - val_accuracy: 0.6639
Epoch 7/1000
229/229 - 40s - loss: 0.4527 - accuracy: 0.7297 - val_loss: 0.6505 - val_accuracy: 0.6697
Epoch 8/1000
229/229 - 40s - loss: 0.4456 - accuracy: 0.7335 - val_loss: 0.7086 - val_accuracy: 0.6539
Epoch 9/1000
229/229 - 40s - loss: 0.4416 - accuracy: 0.7354 - val_loss: 0.7143 - val_accuracy: 0.6645
Epoch 10/1000
229/229 - 39s - loss: 0.4381 - accuracy: 0.7370 - val_loss: